# Sketch-to-KRL Code Generator

This notebook runs a Streamlit web application that converts sketches of robot paths into KUKA Robot Language (KRL) code.

## Setup

First, let's install the required packages and download the application files.

In [ ]:
# Install required packages
!pip install streamlit opencv-python numpy pillow matplotlib scikit-image

In [ ]:
# Create project directory
!mkdir -p sketch_to_krl

In [ ]:
%%writefile sketch_to_krl/app.py
import streamlit as st
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import io
import base64
import os

# Import custom modules
from path_extraction import extract_paths_from_sketch, visualize_paths
from krl_generator import KRLGenerator
from file_utils import get_download_link, create_zip_download
from drawing_canvas import DrawingCanvas
from path_visualization import visualize_robot_path, get_visualization_as_image, overlay_path_on_image

# Set page configuration
st.set_page_config(
    page_title=&quot;Sketch-to-KRL Code Generator&quot;,
    page_icon=&quot;🤖&quot;,
    layout=&quot;wide&quot;
)

# App title and description
st.title(&quot;Sketch-to-KRL Code Generator&quot;)
st.markdown(&quot;&quot;&quot;
Upload or draw a sketch of robot paths, and this app will help you generate KUKA Robot Language (KRL) code.
&quot;&quot;&quot;)

# Global variables to store app state
if 'processed_image' not in st.session_state:
    st.session_state.processed_image = None
if 'extracted_paths' not in st.session_state:
    st.session_state.extracted_paths = None
if 'current_step' not in st.session_state:
    st.session_state.current_step = &quot;upload&quot;  # Possible values: upload, qa, generate, output
if 'start_position' not in st.session_state:
    st.session_state.start_position = &quot;HOME&quot;
if 'motion_types' not in st.session_state:
    st.session_state.motion_types = [&quot;LIN&quot;]
if 'use_coordinates' not in st.session_state:
    st.session_state.use_coordinates = False
if 'krl_code' not in st.session_state:
    st.session_state.krl_code = &quot;&quot;
if 'dat_code' not in st.session_state:
    st.session_state.dat_code = &quot;&quot;
if 'path_smoothing' not in st.session_state:
    st.session_state.path_smoothing = False
if 'extract_dimensions' not in st.session_state:
    st.session_state.extract_dimensions = False
if 'path_simplification' not in st.session_state:
    st.session_state.path_simplification = 50
if 'original_image' not in st.session_state:
    st.session_state.original_image = None

# Function to reset app state
def reset_app():
    st.session_state.processed_image = None
    st.session_state.extracted_paths = None
    st.session_state.current_step = &quot;upload&quot;
    st.session_state.start_position = &quot;HOME&quot;
    st.session_state.motion_types = [&quot;LIN&quot;]
    st.session_state.use_coordinates = False
    st.session_state.krl_code = &quot;&quot;
    st.session_state.dat_code = &quot;&quot;
    st.session_state.path_smoothing = False
    st.session_state.extract_dimensions = False
    st.session_state.path_simplification = 50
    st.session_state.original_image = None

# Main app logic based on current step
if st.session_state.current_step == &quot;upload&quot;:
    # Create two columns for upload and drawing options
    col1, col2 = st.columns(2)
    
    with col1:
        st.header(&quot;Upload Sketch&quot;)
        uploaded_file = st.file_uploader(&quot;Choose an image file&quot;, type=[&quot;jpg&quot;, &quot;jpeg&quot;, &quot;png&quot;])
        
        if uploaded_file is not None:
            # Read and process the uploaded image
            file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
            image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
            
            # Store original image
            st.session_state.original_image = image.copy()
            
            # Process the sketch
            processed_image, paths = extract_paths_from_sketch(image)
            
            # Store in session state
            st.session_state.processed_image = processed_image
            st.session_state.extracted_paths = paths
            
            # Display the processed image
            st.image(processed_image, caption=&quot;Processed Sketch&quot;, use_column_width=True)
            
            # Move to the next step
            if st.button(&quot;Continue with this sketch&quot;):
                st.session_state.current_step = &quot;qa&quot;
                st.experimental_rerun()
    
    with col2:
        st.header(&quot;Draw Sketch&quot;)
        
        # Create drawing canvas
        canvas = DrawingCanvas(width=500, height=500)
        drawn_image = canvas.render()
        
        if drawn_image is not None:
            # Store original image
            st.session_state.original_image = drawn_image.copy()
            
            # Process the drawn image
            processed_image, paths = extract_paths_from_sketch(drawn_image)
            
            # Store in session state
            st.session_state.processed_image = processed_image
            st.session_state.extracted_paths = paths
            
            # Move to the next step
            st.session_state.current_step = &quot;qa&quot;
            st.experimental_rerun()

elif st.session_state.current_step == &quot;qa&quot;:
    # Display the processed image
    if st.session_state.processed_image is not None:
        st.image(st.session_state.processed_image, caption=&quot;Processed Sketch&quot;, width=400)
    
    # Q&A section
    st.header(&quot;Configure Robot Path&quot;)
    
    # Start position
    st.session_state.start_position = st.radio(
        &quot;Select start position:&quot;,
        [&quot;HOME&quot;, &quot;Anywhere&quot;]
    )
    
    # Motion types
    motion_options = st.multiselect(
        &quot;Select motion type(s):&quot;,
        [&quot;LIN&quot;, &quot;PTP&quot;, &quot;CIRC&quot;, &quot;SPLINE&quot;],
        default=[&quot;LIN&quot;]
    )
    
    if motion_options:
        st.session_state.motion_types = motion_options
    
    # Use coordinates
    st.session_state.use_coordinates = st.checkbox(
        &quot;Use exact coordinates from sketch&quot;,
        value=False
    )
    
    # Additional clarifications
    with st.expander(&quot;Additional Options&quot;):
        st.session_state.path_smoothing = st.checkbox(&quot;Enable path smoothing&quot;, value=False)
        st.session_state.extract_dimensions = st.checkbox(&quot;Extract dimensions from sketch&quot;, value=False)
        st.session_state.path_simplification = st.slider(&quot;Path simplification&quot;, 0, 100, 50)
    
    # Navigation buttons
    col1, col2 = st.columns(2)
    with col1:
        if st.button(&quot;Back to Upload&quot;):
            st.session_state.current_step = &quot;upload&quot;
            st.experimental_rerun()
    with col2:
        if st.button(&quot;Generate KRL Code&quot;):
            # Create KRL generator
            krl_gen = KRLGenerator()
            
            # Generate KRL code
            src_code = krl_gen.generate_src_code(
                st.session_state.extracted_paths,
                st.session_state.start_position,
                st.session_state.motion_types,
                st.session_state.use_coordinates
            )
            
            # Generate DAT code
            dat_code = krl_gen.generate_dat_code(st.session_state.use_coordinates)
            
            # Store in session state
            st.session_state.krl_code = src_code
            st.session_state.dat_code = dat_code
            
            # Move to output step
            st.session_state.current_step = &quot;output&quot;
            st.experimental_rerun()

elif st.session_state.current_step == &quot;output&quot;:
    # Create columns for visualization and code
    col1, col2 = st.columns([1, 1])
    
    with col1:
        st.header(&quot;Path Visualization&quot;)
        
        # Create tabs for different visualizations
        viz_tab1, viz_tab2 = st.tabs([&quot;2D Path&quot;, &quot;Overlay on Sketch&quot;])
        
        with viz_tab1:
            # Create 2D visualization
            if st.session_state.extracted_paths:
                fig = visualize_robot_path(
                    st.session_state.extracted_paths,
                    st.session_state.motion_types
                )
                st.pyplot(fig)
        
        with viz_tab2:
            # Create overlay visualization
            if st.session_state.original_image is not None and st.session_state.extracted_paths:
                overlay_image = overlay_path_on_image(
                    st.session_state.original_image,
                    st.session_state.extracted_paths,
                    st.session_state.motion_types
                )
                st.image(overlay_image, caption=&quot;Path Overlay on Original Sketch&quot;, use_column_width=True)
    
    with col2:
        st.header(&quot;Generated KRL Code&quot;)
        
        # Create tabs for SRC and DAT files
        code_tab1, code_tab2 = st.tabs([&quot;SRC File&quot;, &quot;DAT File&quot;])
        
        with code_tab1:
            st.code(st.session_state.krl_code, language=&quot;kotlin&quot;)
        
        with code_tab2:
            st.code(st.session_state.dat_code, language=&quot;kotlin&quot;)
        
        # Download options
        st.subheader(&quot;Download Files&quot;)
        
        # Create download links
        files_dict = {
            &quot;PATH_PROGRAM.src&quot;: st.session_state.krl_code,
            &quot;PATH_PROGRAM.dat&quot;: st.session_state.dat_code
        }
        
        # Individual file downloads
        col1, col2 = st.columns(2)
        with col1:
            st.markdown(get_download_link(st.session_state.krl_code, &quot;PATH_PROGRAM.src&quot;), unsafe_allow_html=True)
        with col2:
            st.markdown(get_download_link(st.session_state.dat_code, &quot;PATH_PROGRAM.dat&quot;), unsafe_allow_html=True)
        
        # ZIP download
        st.markdown(create_zip_download(files_dict), unsafe_allow_html=True)
    
    # Navigation buttons
    col1, col2 = st.columns(2)
    with col1:
        if st.button(&quot;Back to Configuration&quot;):
            st.session_state.current_step = &quot;qa&quot;
            st.experimental_rerun()
    with col2:
        if st.button(&quot;Start Over&quot;):
            reset_app()
            st.experimental_rerun()

# Add a sidebar with information
with st.sidebar:
    st.header(&quot;About&quot;)
    st.info(&quot;&quot;&quot;
    This app converts sketches into KUKA Robot Language (KRL) code.
    
    **Steps:**
    1. Upload or draw a sketch
    2. Configure robot path settings
    3. Generate and download KRL code
    
    **Supported Motion Types:**
    - LIN: Linear motion
    - PTP: Point-to-Point motion
    - CIRC: Circular motion
    - SPLINE: Spline motion
    &quot;&quot;&quot;)
    
    # Add information about KRL
    with st.expander(&quot;About KUKA Robot Language&quot;):
        st.markdown(&quot;&quot;&quot;
        **KUKA Robot Language (KRL)** is the programming language used for KUKA industrial robots.
        
        **Key Components:**
        - **.src files**: Contain the program logic and motion commands
        - **.dat files**: Contain point definitions and other data
        
        **Common Motion Commands:**
        - **PTP**: Point-to-Point motion (fastest path)
        - **LIN**: Linear motion (straight line)
        - **CIRC**: Circular motion (requires auxiliary point)
        - **SPLINE**: Smooth curved motion
        
        **Example KRL Program:**
        ```
        DEF EXAMPLE()
           BAS (#INITMOV,0)
           PTP HOME
           LIN P1
           CIRC P2, P3
           PTP HOME
        END
        ```
        &quot;&quot;&quot;)
    
    # Add a reset button
    if st.button(&quot;Reset Application&quot;):
        reset_app()
        st.experimental_rerun()

In [ ]:
%%writefile sketch_to_krl/path_extraction.py
import cv2
import numpy as np
from skimage import measure
import matplotlib.pyplot as plt

def extract_paths_from_sketch(image):
    &quot;&quot;&quot;
    Extract paths from a sketch image using OpenCV
    
    Args:
        image: Input image as numpy array (BGR format)
        
    Returns:
        processed_image: Visualization of the processed image
        paths: List of extracted paths as coordinate points
    &quot;&quot;&quot;
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Apply adaptive thresholding to handle different lighting conditions
    thresh = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY_INV, 11, 2
    )
    
    # Perform morphological operations to clean up the image
    kernel = np.ones((3, 3), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)
    
    # Skeletonize the image to get thin lines
    skeleton = skeletonize(cleaned)
    
    # Find contours in the skeletonized image
    contours, _ = cv2.findContours(
        skeleton.astype(np.uint8), 
        cv2.RETR_EXTERNAL, 
        cv2.CHAIN_APPROX_NONE
    )
    
    # Create a visualization image
    vis_image = image.copy()
    cv2.drawContours(vis_image, contours, -1, (0, 255, 0), 2)
    
    # Extract paths from contours
    paths = []
    for contour in contours:
        if len(contour) > 10:  # Filter out very small contours
            # Simplify contour to reduce number of points
            epsilon = 0.01 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            
            # Extract points from the contour
            points = []
            for point in approx:
                x, y = point[0]
                points.append((x, y))
            
            paths.append(points)
    
    return vis_image, paths

def skeletonize(img):
    &quot;&quot;&quot;
    Skeletonize a binary image using morphological operations
    &quot;&quot;&quot;
    size = np.size(img)
    skel = np.zeros(img.shape, np.uint8)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    done = False
    
    while not done:
        eroded = cv2.erode(img, element)
        temp = cv2.dilate(eroded, element)
        temp = cv2.subtract(img, temp)
        skel = cv2.bitwise_or(skel, temp)
        img = eroded.copy()
        
        zeros = size - cv2.countNonZero(img)
        if zeros == size:
            done = True
    
    return skel

def visualize_paths(image, paths):
    &quot;&quot;&quot;
    Create a visualization of the extracted paths
    
    Args:
        image: Original image
        paths: List of paths as coordinate points
        
    Returns:
        vis_image: Visualization image with paths drawn
    &quot;&quot;&quot;
    vis_image = image.copy()
    
    # Draw each path with a different color
    colors = [
        (255, 0, 0),    # Red
        (0, 255, 0),    # Green
        (0, 0, 255),    # Blue
        (255, 255, 0),  # Yellow
        (255, 0, 255),  # Magenta
        (0, 255, 255),  # Cyan
    ]
    
    for i, path in enumerate(paths):
        color = colors[i % len(colors)]
        
        # Draw the path
        for j in range(len(path) - 1):
            pt1 = tuple(map(int, path[j]))
            pt2 = tuple(map(int, path[j + 1]))
            cv2.line(vis_image, pt1, pt2, color, 2)
        
        # Draw points
        for point in path:
            x, y = map(int, point)
            cv2.circle(vis_image, (x, y), 5, color, -1)
    
    return vis_image

def extract_dimensions(image, paths):
    &quot;&quot;&quot;
    Attempt to extract dimensions from the sketch
    This is a placeholder for more advanced dimension extraction
    
    Args:
        image: Original image
        paths: List of paths as coordinate points
        
    Returns:
        dimensions: Dictionary of extracted dimensions
    &quot;&quot;&quot;
    # This is a simplified placeholder
    # In a real implementation, we would:
    # 1. Detect text using OCR
    # 2. Associate text with nearby lines
    # 3. Parse dimension values
    
    dimensions = {}
    
    # For now, just calculate the bounding box of each path
    for i, path in enumerate(paths):
        if path:
            xs = [p[0] for p in path]
            ys = [p[1] for p in path]
            width = max(xs) - min(xs)
            height = max(ys) - min(ys)
            dimensions[f&quot;path_{i}&quot;] = {
                &quot;width&quot;: width,
                &quot;height&quot;: height,
                &quot;length&quot;: sum(
                    np.sqrt((path[j][0] - path[j-1][0])**2 + (path[j][1] - path[j-1][1])**2)
                    for j in range(1, len(path))
                )
            }
    
    return dimensions

In [ ]:
%%writefile sketch_to_krl/krl_generator.py
class KRLGenerator:
    &quot;&quot;&quot;
    Class for generating KUKA Robot Language (KRL) code from paths
    &quot;&quot;&quot;
    
    def __init__(self, program_name=&quot;PATH_PROGRAM&quot;):
        &quot;&quot;&quot;
        Initialize the KRL generator
        
        Args:
            program_name: Name of the KRL program
        &quot;&quot;&quot;
        self.program_name = program_name
        self.points = []
    
    def generate_src_code(self, paths, start_position, motion_types, use_coordinates=False):
        &quot;&quot;&quot;
        Generate KRL source code (.src file)
        
        Args:
            paths: List of paths as coordinate points
            start_position: Starting position (&quot;HOME&quot; or &quot;Anywhere&quot;)
            motion_types: List of motion types to use (LIN, PTP, CIRC, SPLINE)
            use_coordinates: Whether to use exact coordinates from the sketch
            
        Returns:
            src_code: Generated KRL source code
        &quot;&quot;&quot;
        # Start with the program header
        src_code = f&quot;DEF {self.program_name}()\n&quot;
        src_code += &quot;   BAS (#INITMOV,0)\n&quot;
        
        # Add start position
        if start_position == &quot;HOME&quot;:
            src_code += &quot;   PTP HOME\n&quot;
        else:
            src_code += &quot;   PTP P0\n&quot;
        
        # Reset points list
        self.points = []
        
        # Generate motion commands based on extracted paths
        point_index = 1
        
        for path_idx, path in enumerate(paths):
            if not path:
                continue
                
            # Skip paths that are too short
            if len(path) < 3:
                continue
            
            # Distribute motion types along the path
            if not motion_types:
                motion_types = [&quot;LIN&quot;]  # Default to LIN if none selected
            
            # Process points in the path
            i = 0
            while i < len(path):
                motion_type = motion_types[i % len(motion_types)]
                
                # Store the point for DAT file generation
                self.points.append(path[i])
                
                if motion_type == &quot;CIRC&quot; and i < len(path) - 2:
                    # CIRC requires two points: auxiliary and end point
                    aux_point = path[i + 1]
                    end_point = path[i + 2]
                    
                    # Store the auxiliary point
                    self.points.append(aux_point)
                    
                    src_code += f&quot;   CIRC P{point_index}, P{point_index + 1}\n&quot;
                    point_index += 2
                    i += 3  # Skip the next two points as they're used for the CIRC
                
                elif motion_type == &quot;SPLINE&quot; and i < len(path) - 3:
                    # Start SPLINE block
                    src_code += &quot;   SPLINE\n&quot;
                    
                    # Add SPL points
                    spline_points = min(4, len(path) - i)
                    for j in range(spline_points):
                        # Store the point
                        if i + j < len(path):
                            self.points.append(path[i + j])
                            src_code += f&quot;      SPL P{point_index}\n&quot;
                            point_index += 1
                    
                    # End SPLINE block
                    src_code += &quot;   ENDSPLINE\n&quot;
                    i += spline_points  # Skip points used in the SPLINE
                
                else:
                    # LIN or PTP
                    src_code += f&quot;   {motion_type} P{point_index}\n&quot;
                    point_index += 1
                    i += 1
        
        # Return to home position
        src_code += &quot;   PTP HOME\n&quot;
        
        # End program
        src_code += &quot;END\n&quot;
        
        return src_code
    
    def generate_dat_code(self, use_coordinates=False):
        &quot;&quot;&quot;
        Generate KRL data file (.dat file)
        
        Args:
            use_coordinates: Whether to use exact coordinates from the sketch
            
        Returns:
            dat_code: Generated KRL data code
        &quot;&quot;&quot;
        # Generate DAT file header
        dat_code = f&quot;&ACCESS RVP\n&REL 1\n&PARAM TEMPLATE = C_PTP\n&PARAM EDITMASK = *\nDEFDAT {self.program_name}\n\n&quot;
        dat_code += &quot;DECL E6POS XHOME={X 0.0,Y 0.0,Z 0.0,A 0.0,B 0.0,C 0.0}\n&quot;
        
        # Add point definitions
        for i, point in enumerate(self.points):
            x, y = point
            
            # Scale coordinates to a reasonable robot workspace (mm)
            # Assuming the sketch is in pixel coordinates
            x_scaled = (x / 500) * 1000  # Scale to 0-1000mm range
            y_scaled = (y / 500) * 1000
            z = 100  # Fixed Z height for simplicity
            
            # Add some variation to Z for visual interest
            if i % 3 == 0:
                z = 120
            elif i % 3 == 1:
                z = 100
            else:
                z = 80
            
            dat_code += f&quot;DECL E6POS P{i+1}={{X {x_scaled:.1f},Y {y_scaled:.1f},Z {z:.1f},A 0.0,B 90.0,C 0.0}}\n&quot;
        
        dat_code += &quot;ENDDAT\n&quot;
        
        return dat_code

In [ ]:
%%writefile sketch_to_krl/file_utils.py
import base64
import io
import zipfile
import streamlit as st

def get_download_link(file_content, file_name):
    &quot;&quot;&quot;
    Create a download link for a text file
    
    Args:
        file_content: Content of the file as string
        file_name: Name of the file
        
    Returns:
        download_link: HTML link for downloading the file
    &quot;&quot;&quot;
    b64 = base64.b64encode(file_content.encode()).decode()
    return f'<a href=&quot;data:file/txt;base64,{b64}&quot; download=&quot;{file_name}&quot;>Download {file_name}</a>'

def get_image_download_link(img, file_name):
    &quot;&quot;&quot;
    Create a download link for an image
    
    Args:
        img: PIL Image object
        file_name: Name of the file
        
    Returns:
        download_link: HTML link for downloading the image
    &quot;&quot;&quot;
    buffered = io.BytesIO()
    img.save(buffered, format=&quot;PNG&quot;)
    img_str = base64.b64encode(buffered.getvalue()).decode()
    return f'<a href=&quot;data:file/png;base64,{img_str}&quot; download=&quot;{file_name}&quot;>Download {file_name}</a>'

def create_zip_download(files_dict):
    &quot;&quot;&quot;
    Create a download link for a zip file containing multiple files
    
    Args:
        files_dict: Dictionary of {filename: content}
        
    Returns:
        download_link: HTML link for downloading the zip file
    &quot;&quot;&quot;
    zip_buffer = io.BytesIO()
    
    with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        for file_name, file_content in files_dict.items():
            zip_file.writestr(file_name, file_content)
    
    zip_buffer.seek(0)
    b64 = base64.b64encode(zip_buffer.getvalue()).decode()
    
    return f'<a href=&quot;data:application/zip;base64,{b64}&quot; download=&quot;krl_program.zip&quot;>Download All Files (ZIP)</a>'

def save_uploaded_file(uploaded_file):
    &quot;&quot;&quot;
    Save an uploaded file to disk
    
    Args:
        uploaded_file: Streamlit UploadedFile object
        
    Returns:
        file_path: Path to the saved file
    &quot;&quot;&quot;
    import os
    
    # Create uploads directory if it doesn't exist
    if not os.path.exists(&quot;uploads&quot;):
        os.makedirs(&quot;uploads&quot;)
    
    # Save the file
    file_path = os.path.join(&quot;uploads&quot;, uploaded_file.name)
    with open(file_path, &quot;wb&quot;) as f:
        f.write(uploaded_file.getbuffer())
    
    return file_path

In [ ]:
%%writefile sketch_to_krl/drawing_canvas.py
import streamlit as st
import numpy as np
from PIL import Image, ImageDraw
import cv2
import base64
from io import BytesIO

class DrawingCanvas:
    &quot;&quot;&quot;
    Interactive drawing canvas for Streamlit
    &quot;&quot;&quot;
    
    def __init__(self, width=500, height=500, background_color=(255, 255, 255)):
        &quot;&quot;&quot;
        Initialize the drawing canvas
        
        Args:
            width: Canvas width in pixels
            height: Canvas height in pixels
            background_color: Canvas background color as RGB tuple
        &quot;&quot;&quot;
        self.width = width
        self.height = height
        self.background_color = background_color
        
        # Initialize canvas state if not already in session state
        if 'canvas_image' not in st.session_state:
            self.reset_canvas()
        
        if 'drawing_mode' not in st.session_state:
            st.session_state.drawing_mode = &quot;line&quot;
        
        if 'drawing_color' not in st.session_state:
            st.session_state.drawing_color = (0, 0, 0)
        
        if 'drawing_thickness' not in st.session_state:
            st.session_state.drawing_thickness = 3
        
        if 'drawing_points' not in st.session_state:
            st.session_state.drawing_points = []
    
    def reset_canvas(self):
        &quot;&quot;&quot;Reset the canvas to a blank state&quot;&quot;&quot;
        st.session_state.canvas_image = Image.new('RGB', (self.width, self.height), self.background_color)
        st.session_state.canvas_draw = ImageDraw.Draw(st.session_state.canvas_image)
        st.session_state.drawing_points = []
    
    def render(self):
        &quot;&quot;&quot;
        Render the drawing canvas in the Streamlit app
        
        Returns:
            image: The drawn image as numpy array if submitted, None otherwise
        &quot;&quot;&quot;
        st.markdown(&quot;### Drawing Canvas&quot;)
        st.markdown(&quot;Draw your robot path on the canvas below.&quot;)
        
        # Drawing tools
        col1, col2, col3 = st.columns(3)
        
        with col1:
            st.session_state.drawing_mode = st.selectbox(
                &quot;Drawing Mode&quot;,
                [&quot;line&quot;, &quot;rectangle&quot;, &quot;circle&quot;, &quot;freehand&quot;]
            )
        
        with col2:
            color_options = {
                &quot;Black&quot;: (0, 0, 0),
                &quot;Red&quot;: (255, 0, 0),
                &quot;Green&quot;: (0, 255, 0),
                &quot;Blue&quot;: (0, 0, 255)
            }
            color_choice = st.selectbox(&quot;Color&quot;, list(color_options.keys()))
            st.session_state.drawing_color = color_options[color_choice]
        
        with col3:
            st.session_state.drawing_thickness = st.slider(&quot;Thickness&quot;, 1, 10, 3)
        
        # Canvas area
        canvas_col, controls_col = st.columns([3, 1])
        
        with canvas_col:
            # Display the canvas
            st.image(st.session_state.canvas_image, caption=&quot;Drawing Canvas&quot;, use_column_width=True)
            
            # Create a placeholder for the canvas
            canvas_placeholder = st.empty()
            
            # Display the current canvas
            canvas_placeholder.image(st.session_state.canvas_image, caption=&quot;Drawing Canvas&quot;, use_column_width=True)
        
        with controls_col:
            # Drawing controls based on mode
            if st.session_state.drawing_mode == &quot;line&quot;:
                st.markdown(&quot;#### Draw Line&quot;)
                start_x = st.slider(&quot;Start X&quot;, 0, self.width, self.width // 4)
                start_y = st.slider(&quot;Start Y&quot;, 0, self.height, self.height // 4)
                end_x = st.slider(&quot;End X&quot;, 0, self.width, 3 * self.width // 4)
                end_y = st.slider(&quot;End Y&quot;, 0, self.height, 3 * self.height // 4)
                
                if st.button(&quot;Add Line&quot;):
                    st.session_state.canvas_draw.line(
                        [(start_x, start_y), (end_x, end_y)],
                        fill=st.session_state.drawing_color,
                        width=st.session_state.drawing_thickness
                    )
                    # Add points to the drawing points list
                    st.session_state.drawing_points.append((start_x, start_y))
                    st.session_state.drawing_points.append((end_x, end_y))
                    # Update the canvas display
                    canvas_placeholder.image(st.session_state.canvas_image, caption=&quot;Drawing Canvas&quot;, use_column_width=True)
            
            elif st.session_state.drawing_mode == &quot;rectangle&quot;:
                st.markdown(&quot;#### Draw Rectangle&quot;)
                start_x = st.slider(&quot;Left&quot;, 0, self.width, self.width // 4)
                start_y = st.slider(&quot;Top&quot;, 0, self.height, self.height // 4)
                end_x = st.slider(&quot;Right&quot;, 0, self.width, 3 * self.width // 4)
                end_y = st.slider(&quot;Bottom&quot;, 0, self.height, 3 * self.height // 4)
                
                if st.button(&quot;Add Rectangle&quot;):
                    st.session_state.canvas_draw.rectangle(
                        [(start_x, start_y), (end_x, end_y)],
                        outline=st.session_state.drawing_color,
                        width=st.session_state.drawing_thickness
                    )
                    # Add points to the drawing points list (corners of rectangle)
                    st.session_state.drawing_points.extend([
                        (start_x, start_y),
                        (end_x, start_y),
                        (end_x, end_y),
                        (start_x, end_y),
                        (start_x, start_y)  # Close the rectangle
                    ])
                    # Update the canvas display
                    canvas_placeholder.image(st.session_state.canvas_image, caption=&quot;Drawing Canvas&quot;, use_column_width=True)
            
            elif st.session_state.drawing_mode == &quot;circle&quot;:
                st.markdown(&quot;#### Draw Circle&quot;)
                center_x = st.slider(&quot;Center X&quot;, 0, self.width, self.width // 2)
                center_y = st.slider(&quot;Center Y&quot;, 0, self.height, self.height // 2)
                radius = st.slider(&quot;Radius&quot;, 5, min(self.width, self.height) // 2, 50)
                
                if st.button(&quot;Add Circle&quot;):
                    st.session_state.canvas_draw.ellipse(
                        [(center_x - radius, center_y - radius), 
                         (center_x + radius, center_y + radius)],
                        outline=st.session_state.drawing_color,
                        width=st.session_state.drawing_thickness
                    )
                    # Add points approximating a circle
                    num_points = 20
                    for i in range(num_points + 1):
                        angle = 2 * np.pi * i / num_points
                        x = center_x + radius * np.cos(angle)
                        y = center_y + radius * np.sin(angle)
                        st.session_state.drawing_points.append((int(x), int(y)))
                    # Update the canvas display
                    canvas_placeholder.image(st.session_state.canvas_image, caption=&quot;Drawing Canvas&quot;, use_column_width=True)
            
            elif st.session_state.drawing_mode == &quot;freehand&quot;:
                st.markdown(&quot;#### Freehand Drawing&quot;)
                st.markdown(&quot;Coming soon: Interactive freehand drawing&quot;)
                st.markdown(&quot;For now, use the other drawing modes.&quot;)
            
            # Clear button
            if st.button(&quot;Clear Canvas&quot;):
                self.reset_canvas()
                canvas_placeholder.image(st.session_state.canvas_image, caption=&quot;Drawing Canvas&quot;, use_column_width=True)
        
        # Submit button
        if st.button(&quot;Use This Drawing&quot;):
            # Convert to numpy array for OpenCV processing
            return np.array(st.session_state.canvas_image)
        
        return None
    
    def get_drawing_points(self):
        &quot;&quot;&quot;
        Get the points that were drawn on the canvas
        
        Returns:
            points: List of (x, y) coordinate tuples
        &quot;&quot;&quot;
        return st.session_state.drawing_points
    
    def get_image_as_base64(self):
        &quot;&quot;&quot;
        Get the canvas image as a base64 encoded string
        
        Returns:
            base64_image: Base64 encoded image string
        &quot;&quot;&quot;
        buffered = BytesIO()
        st.session_state.canvas_image.save(buffered, format=&quot;PNG&quot;)
        img_str = base64.b64encode(buffered.getvalue()).decode()
        return img_str

In [ ]:
%%writefile sketch_to_krl/path_visualization.py
import matplotlib.pyplot as plt
import numpy as np
import io
import base64
from PIL import Image

def visualize_robot_path(paths, motion_types, figsize=(8, 6)):
    &quot;&quot;&quot;
    Create a 2D visualization of the robot path
    
    Args:
        paths: List of paths as coordinate points
        motion_types: List of motion types used
        figsize: Figure size as (width, height) tuple
        
    Returns:
        fig: Matplotlib figure object
    &quot;&quot;&quot;
    fig, ax = plt.subplots(figsize=figsize)
    
    # Define colors for different motion types
    motion_colors = {
        &quot;LIN&quot;: &quot;blue&quot;,
        &quot;PTP&quot;: &quot;red&quot;,
        &quot;CIRC&quot;: &quot;green&quot;,
        &quot;SPLINE&quot;: &quot;purple&quot;
    }
    
    # Plot each path
    for path_idx, path in enumerate(paths):
        if not path:
            continue
        
        # Extract x and y coordinates
        xs = [p[0] for p in path]
        ys = [p[1] for p in path]
        
        # Plot the path
        ax.plot(xs, ys, 'k-', alpha=0.3, linewidth=1)
        
        # Plot points with motion type colors
        for i, (x, y) in enumerate(path):
            motion_type = motion_types[i % len(motion_types)]
            color = motion_colors.get(motion_type, &quot;black&quot;)
            
            # Plot the point
            ax.scatter(x, y, color=color, s=50, zorder=10)
            
            # Add point label
            ax.text(x + 5, y + 5, f&quot;P{i+1}&quot;, fontsize=8)
    
    # Add legend
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=motion_type)
        for motion_type, color in motion_colors.items()
        if motion_type in motion_types
    ]
    ax.legend(handles=legend_elements, loc='upper right')
    
    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('Robot Path Visualization')
    
    # Invert y-axis to match image coordinates
    ax.invert_yaxis()
    
    # Set equal aspect ratio
    ax.set_aspect('equal')
    
    # Add grid
    ax.grid(True, linestyle='--', alpha=0.7)
    
    return fig

def get_visualization_as_image(fig):
    &quot;&quot;&quot;
    Convert a matplotlib figure to a PIL Image
    
    Args:
        fig: Matplotlib figure object
        
    Returns:
        image: PIL Image object
    &quot;&quot;&quot;
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight')
    buf.seek(0)
    img = Image.open(buf)
    return img

def get_visualization_as_base64(fig):
    &quot;&quot;&quot;
    Convert a matplotlib figure to a base64 encoded string
    
    Args:
        fig: Matplotlib figure object
        
    Returns:
        base64_image: Base64 encoded image string
    &quot;&quot;&quot;
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight')
    buf.seek(0)
    img_str = base64.b64encode(buf.getvalue()).decode()
    return img_str

def overlay_path_on_image(image, paths, motion_types):
    &quot;&quot;&quot;
    Overlay the robot path on the original image
    
    Args:
        image: Original image as numpy array
        paths: List of paths as coordinate points
        motion_types: List of motion types used
        
    Returns:
        overlay_image: Image with path overlay
    &quot;&quot;&quot;
    # Create a copy of the image
    overlay = image.copy()
    
    # Define colors for different motion types (BGR format for OpenCV)
    motion_colors = {
        &quot;LIN&quot;: (255, 0, 0),    # Blue
        &quot;PTP&quot;: (0, 0, 255),    # Red
        &quot;CIRC&quot;: (0, 255, 0),   # Green
        &quot;SPLINE&quot;: (255, 0, 255)  # Purple
    }
    
    # Draw each path
    for path_idx, path in enumerate(paths):
        if not path:
            continue
        
        # Draw lines connecting points
        for i in range(len(path) - 1):
            pt1 = tuple(map(int, path[i]))
            pt2 = tuple(map(int, path[i + 1]))
            
            # Get motion type for this segment
            motion_type = motion_types[i % len(motion_types)]
            color = motion_colors.get(motion_type, (0, 0, 0))
            
            # Draw line
            import cv2
            cv2.line(overlay, pt1, pt2, color, 2)
        
        # Draw points
        for i, point in enumerate(path):
            x, y = map(int, point)
            
            # Get motion type for this point
            motion_type = motion_types[i % len(motion_types)]
            color = motion_colors.get(motion_type, (0, 0, 0))
            
            # Draw circle
            import cv2
            cv2.circle(overlay, (x, y), 5, color, -1)
            
            # Add point label
            cv2.putText(overlay, f&quot;P{i+1}&quot;, (x + 5, y + 5), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    
    return overlay

In [ ]:
%%writefile sketch_to_krl/requirements.txt
streamlit==1.30.0
opencv-python==4.8.0.76
numpy==1.24.3
pillow==10.0.0
matplotlib==3.7.2
scikit-image==0.21.0

## Run the Streamlit App

Now let's run the Streamlit app using localtunnel to make it accessible.

In [ ]:
# Install localtunnel
!pip install pyngrok

In [ ]:
# Run the Streamlit app
import os
from pyngrok import ngrok

# Set up ngrok
public_url = ngrok.connect(port=8501)
print(f&quot;Public URL: {public_url}&quot;)

# Change to the app directory
os.chdir('sketch_to_krl')

# Run the Streamlit app
!streamlit run app.py &>/dev/null &

# Keep the notebook running
import IPython
from IPython.display import display, HTML
display(HTML(f&quot;&quot;&quot;
<div style=&quot;background-color:#4CAF50; padding: 10px; border-radius: 5px;&quot;>
  <h3 style=&quot;color: white;&quot;>App is running!</h3>
  <p style=&quot;color: white;&quot;>Access your Sketch-to-KRL Code Generator at: <a href=&quot;{public_url}&quot; target=&quot;_blank&quot; style=&quot;color: white; text-decoration: underline;&quot;>{public_url}</a></p>
</div>
&quot;&quot;&quot;))

## Hugging Face Spaces Deployment

To deploy this app on Hugging Face Spaces, follow these steps:

1. Create a new Space on Hugging Face (https://huggingface.co/spaces)
2. Select Streamlit as the SDK
3. Upload all the files in the `sketch_to_krl` directory to your Space
4. Make sure to include the `requirements.txt` file
5. The app should automatically deploy and be accessible via the Hugging Face URL

## Example Test Sketches

Here are some example sketches you can use to test the application:

In [ ]:
# Create a simple test sketch
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

# Create a blank image
img = np.ones((500, 500, 3), dtype=np.uint8) * 255

# Draw a simple path
cv2.line(img, (100, 100), (400, 100), (0, 0, 0), 5)
cv2.line(img, (400, 100), (400, 400), (0, 0, 0), 5)
cv2.line(img, (400, 400), (100, 400), (0, 0, 0), 5)
cv2.line(img, (100, 400), (100, 100), (0, 0, 0), 5)

# Add a diagonal line
cv2.line(img, (100, 100), (400, 400), (0, 0, 0), 5)

# Save the image
cv2.imwrite('sketch_to_krl/test_sketch_square.png', img)

# Create a circular path
img2 = np.ones((500, 500, 3), dtype=np.uint8) * 255
cv2.circle(img2, (250, 250), 200, (0, 0, 0), 5)

# Save the image
cv2.imwrite('sketch_to_krl/test_sketch_circle.png', img2)

# Create a complex path
img3 = np.ones((500, 500, 3), dtype=np.uint8) * 255
pil_img = Image.fromarray(img3)
draw = ImageDraw.Draw(pil_img)

# Draw a star
points = []
center_x, center_y = 250, 250
outer_radius = 200
inner_radius = 100
num_points = 5

for i in range(num_points * 2):
    angle = np.pi / num_points * i
    radius = outer_radius if i % 2 == 0 else inner_radius
    x = center_x + radius * np.cos(angle)
    y = center_y + radius * np.sin(angle)
    points.append((x, y))

# Close the shape
points.append(points[0])

# Draw the star
draw.line(points, fill=(0, 0, 0), width=5)

# Convert back to numpy array
img3 = np.array(pil_img)

# Save the image
cv2.imwrite('sketch_to_krl/test_sketch_star.png', img3)

# Display the test sketches
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title('Square with Diagonal')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
plt.title('Circle')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))
plt.title('Star')
plt.axis('off')

plt.tight_layout()
plt.show()